## Results Analysis

In [ ]:
import pandas as pd
import numpy as np

llm_exp_result_path = 'llm_exp/full_exp_merged.json'
df_llm = pd.read_json(llm_exp_result_path)

tts_exp_kokoro_result_path = 'tts_exp/tts_exp_l4_kokoro/full_results.json'
tts_exp_piper_result_path = 'tts_exp/tts_exp_l4_piper/full_results.json'
df_kokoro = pd.read_json(tts_exp_kokoro_result_path)
df_piper = pd.read_json(tts_exp_piper_result_path)

assert np.all(df_kokoro.columns == df_piper.columns)

# df_piper['inference_time'] = -df_piper['inference_time']
# df_kokoro['inference_time'] = -df_kokoro['inference_time']

In [ ]:
kokoro_durations = list(map(lambda x: x['duration_s'], df_kokoro['audio_metrics'].to_list()))
piper_durations = list(map(lambda x: x['duration_s'], df_piper['audio_metrics'].to_list()))
df_kokoro['audio_duration'] = kokoro_durations
df_piper['audio_duration'] = piper_durations
df_piper.columns
tts_metrics = ['wer', 'inference_time', 'audio_duration']

In [ ]:
df_kokoro = df_kokoro.rename(columns=lambda c: c + "_kokoro" if c in tts_metrics else c)
df_piper = df_piper.rename(columns=lambda c: c + "_piper" if c in tts_metrics else c)

In [ ]:
df_llm.columns

In [ ]:
df_kokoro.columns
df_piper.columns
tts_metrics_kokoro = list(map(lambda x: x+'_kokoro', tts_metrics))
tts_metrics_piper = list(map(lambda x: x+'_piper', tts_metrics))
df_llm[tts_metrics_kokoro] = df_kokoro[tts_metrics_kokoro]
df_llm[tts_metrics_piper] = df_piper[tts_metrics_piper]

In [ ]:
df_llm['inference_time_pipeline_piper'] = df_llm['inference_seconds'] + df_llm['inference_time_piper']
df_llm['inference_time_pipeline_kokoro'] = df_llm['inference_seconds'] + df_llm['inference_time_kokoro']

In [ ]:
df = df_llm
df['dialogue_words_count'] = df.loc[:, 'output_text'].map(lambda x: len(x['dialogue'].replace('HOST:', '').replace('GUEST:', '').strip().split(' ')))
df = df.rename(columns={'inference_seconds': 'inference_time_llm'})
df.columns

In [ ]:
import seaborn as sns
features_mean = ['few_shot_nr', 'max_tokens','inference_time_llm', 'total_time_seconds', 'prompt_token_count', 'output_token_count',
       'total_tokens','audio_duration_kokoro','audio_duration_piper', 'dialogue_words_count']
df_aggr = df[features_mean].groupby(['few_shot_nr', 'max_tokens'], as_index=False).mean()
df_durations = pd.DataFrame({'audio_duration': np.concatenate([df_aggr['audio_duration_kokoro'].to_numpy(), df_aggr['audio_duration_piper'].to_numpy()]), 
                             'words_count': np.concatenate([df_aggr['dialogue_words_count'].to_numpy(), df_aggr['dialogue_words_count'].to_numpy()]),
                             'label': 20*['kokoro'] + 20*['piper'],
                             'max_tokens': np.concatenate([df_aggr['max_tokens'].to_numpy(), df_aggr['max_tokens'].to_numpy()])})
sns.scatterplot(df_durations, x='audio_duration', y='words_count', hue='label', style='max_tokens')
lin_gen = lambda m, q: lambda x: m*x + q
m_kok,q_kok = np.polyfit(df_aggr['audio_duration_kokoro'], df_aggr['dialogue_words_count'], deg=1)
m_pip,q_pip = np.polyfit(df_aggr['audio_duration_piper'], df_aggr['dialogue_words_count'], deg=1)
x_range = np.array([0,100])
sns.lineplot(x=x_range, y=lin_gen(m_kok, q_kok)(x_range))
ax = sns.lineplot(x=x_range, y=lin_gen(m_pip, q_pip)(x_range))
sns.lineplot(x=[0,100], y=[150, 150], linestyle='--', c='green')
sns.lineplot(x=[0,100], y=[200, 200], linestyle='--', c='green')
ax.set_title(f"Kokoro WPM: {np.round(m_kok*60, 2)}, Piper WPM: {np.round(m_pip*60, 2)}")

In [ ]:
# tried to define a judge score but in the end it wasn't meaningful
judge_identifiers = ['_mistral_g5', '_mistral_g10', '_llama_g5', '_llama_g10']
coeff = 0.01
target_word_count = 175

for judge_id in judge_identifiers:
  avg_score = (df[f'coherence_score{judge_id}'] + df[f'relevance_score{judge_id}'] + df[f'compliance_score{judge_id}']) / 3
  penalty = coeff * (target_word_count - df['dialogue_words_count']).map(lambda x: max(0, x))
  df[f'composite_score{judge_id}'] = avg_score - penalty
  df[f'composite_score{judge_id}'] = df[f'composite_score{judge_id}'].map(lambda x: max(1.0, x))

In [ ]:
import pandas as pd
from itertools import product

# The experiment varying configurations
few_shot_nrs = np.unique(df['few_shot_nr'])
max_tokens = np.unique(df['max_tokens'])
configurations = list(product(few_shot_nrs, max_tokens))

# results : {(few_shot_nr, max_tokens): df with index=(mean,std) and columns=feeatures}
results = {}

features = ['prompt_token_count', 'output_token_count', 'inference_time_llm',
       'total_tokens', 'relevance_score_llama_g5', 'coherence_score_llama_g5',
       'compliance_score_llama_g5', 'relevance_score_llama_g10',
       'coherence_score_llama_g10', 'compliance_score_llama_g10',
       'relevance_score_mistral_g5', 'coherence_score_mistral_g5',
       'compliance_score_mistral_g5', 'relevance_score_mistral_g10',
       'coherence_score_mistral_g10', 'compliance_score_mistral_g10',
       'wer_piper', 'inference_time_piper', 'audio_duration_piper',
       'inference_time_pipeline_piper',
       'wer_kokoro', 'inference_time_kokoro', 'audio_duration_kokoro',
       'inference_time_pipeline_kokoro', 'composite_score_mistral_g5',
       'composite_score_mistral_g10', 'composite_score_llama_g5',
       'composite_score_llama_g10']

for conf in configurations:
    df_conf = df[
        (df['few_shot_nr'] == conf[0]) &
        (df['max_tokens'] == conf[1])
    ]
    # aggregate rows by doing mean and std
    stats = df_conf[features].agg(['mean', 'std'])
    results[conf] = stats

# Flatten the results dict to a DataFrame
plot_data = []

for conf, stats in results.items():
    for feature in features:
        plot_data.append({
            'few_shot_nr': conf[0],
            'max_tokens': conf[1],
            'feature': feature,
            'mean': stats.loc['mean', feature],
            'std': stats.loc['std', feature]
        })

df_plot = pd.DataFrame(plot_data)

## Plots

In [ ]:
import matplotlib.pyplot as plt

def plot_metric_vs_few_shot(
    df_plot,
    metric,
    x='few_shot_nr',
    style='max_tokens',
    ax=None,
    y_range=None,
    err=None,
    title_addon="(means on 10 topics)"
):
    # Filter the metric
    df_metric = df_plot[df_plot['feature'] == metric]

    if ax is None:
        _, ax = plt.subplots(figsize=(10, 6))

    for val in sorted(df_metric[style].unique()):
        df_val = df_metric[df_metric[style] == val]

        ax.plot(
            df_val[x],
            df_val['mean'],
            marker='o',
            label=f"{style}={val}"
        )

        if err == 'band':
            ax.fill_between(
                df_val[x],
                df_val['mean'] - df_val['std'],
                df_val['mean'] + df_val['std'],
                alpha=0.2
            )

        elif err == 'bar':
            ax.errorbar(
                df_val[x],
                df_val['mean'],
                yerr=df_val['std'],
                fmt='none',
                capsize=5,
                alpha=0.8
            )

    if y_range is not None:
        ax.set_ylim(y_range)

    ax.set_title(
        f"{metric.replace('_', ' ').title()} vs "
        f"{x.replace('_', ' ').title()} {title_addon}"
    )
    ax.set_xlabel(x.replace('_', ' ').title())
    ax.set_ylabel(metric.replace('_', ' ').title())
    ax.legend(title=style)
    ax.grid(True)

    return ax


In [ ]:
ax = plot_metric_vs_few_shot(df_plot, x='max_tokens', style='few_shot_nr', metric='audio_duration_piper')
wpm = 150
target_words = 200
target_duration_s = target_words/wpm * 60
ax.plot([0, 1024], [target_duration_s, target_duration_s], linestyle='--', c='black')

In [ ]:
plot_metric_vs_few_shot(df_plot, 'inference_time_llm', err='band')

In [ ]:
df_plot_sub = df_plot[df_plot['max_tokens'].isin([256, 512])]
fig, axes = plt.subplots(1, 2, figsize=(15,5))
plot_metric_vs_few_shot(df_plot_sub, 'inference_time_pipeline_kokoro', err='band', ax=axes[0], title_addon="")
plot_metric_vs_few_shot(df_plot_sub, 'inference_time_pipeline_piper', err='band', ax=axes[1], title_addon="")

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15,5))
plot_metric_vs_few_shot(df_plot_sub, 'wer_kokoro', err="band", ax=axes[0], title_addon="", y_range=[0,0.3])
plot_metric_vs_few_shot(df_plot_sub, 'wer_piper', err="band", ax=axes[1], title_addon="", y_range=[0,0.3])

## LLM judge scores

In [ ]:
fig, axes = plt.subplots(2,2, figsize=(10, 5), layout='constrained')
y_ranges = [[1,5], [1,10]]
cmap='viridis'

for x, judge_id in enumerate(judge_identifiers):
  df_s = df_plot[df_plot['feature']==f'composite_score{judge_id}'].reset_index()
  hm_s = df_s['mean'].to_numpy().reshape(4, -1)
  ax = axes[x//2,x%2]
  sns.heatmap(hm_s, ax=ax, vmin=y_ranges[x%2][0], vmax=y_ranges[x%2][1], cmap=cmap, 
            yticklabels=df_s.loc[0::5,'few_shot_nr'],
            xticklabels=df_s.loc[:4, 'max_tokens'], annot=True)
  ax.set_title(f"Judge{judge_id.replace('_', ' ')}", fontsize=11)

In [ ]:
import seaborn as sns
judge_identifier = '_mistral_g5'
# judge_identifier = '_mistral_g10'
# judge_identifier = '_llama_g5'
# judge_identifier = '_llama_g10'

y_range = [1, 6]
cmap='viridis'

def row_heat_map(judge_identifier, y_range, row_idx):
  df_rs = df_plot[df_plot['feature']==f'relevance_score{judge_identifier}'].reset_index()
  hm_rs = df_rs['mean'].to_numpy().reshape(4, -1)

  df_cs = df_plot[df_plot['feature']==f'coherence_score{judge_identifier}'].reset_index()
  hm_cs = df_cs['mean'].to_numpy().reshape(4, -1)

  df_comp_s = df_plot[df_plot['feature']==f'compliance_score{judge_identifier}'].reset_index()
  hm_comp_s = df_comp_s['mean'].to_numpy().reshape(4, -1)

  sns.heatmap(hm_rs, ax=axes[row_idx,0], vmin=y_range[0], vmax=y_range[1], cmap=cmap, 
            yticklabels=df_rs.loc[0::5,'few_shot_nr'],
            xticklabels=df_rs.loc[:4, 'max_tokens'], annot=True)
  sns.heatmap(hm_cs, ax=axes[row_idx,1], vmin=y_range[0], vmax=y_range[1], cmap=cmap, 
              yticklabels=df_cs.loc[0::5,'few_shot_nr'],
              xticklabels=df_cs.loc[:4, 'max_tokens'], annot=True)
  sns.heatmap(hm_comp_s, ax=axes[row_idx,2], vmin=y_range[0], vmax=y_range[1], cmap=cmap, 
              yticklabels=df_comp_s.loc[0::5,'few_shot_nr'],
              xticklabels=df_comp_s.loc[:4, 'max_tokens'], annot=True)
  
  axes[row_idx,0].set_title('relevance score')
  axes[row_idx,0].set_ylabel('few shot nr')
  axes[row_idx,0].set_xlabel('max token len')

  axes[row_idx,1].set_title('coherence score')
  axes[row_idx,1].set_ylabel('few shot nr')
  axes[row_idx,1].set_xlabel('max token len')

  axes[row_idx,2].set_title('compliance score')
  axes[row_idx,2].set_ylabel('few shot nr')
  axes[row_idx,2].set_xlabel('max token len')

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(10,5), layout='constrained', sharex=True, sharey=True)
# plot_metric_vs_few_shot(df_plot, f'relevance_score{judge_identifier}', y_range=y_range, ax=axes[0,0], title_addon="")
# plot_metric_vs_few_shot(df_plot, f'coherence_score{judge_identifier}', y_range=y_range, ax=axes[0,1], title_addon="")
# plot_metric_vs_few_shot(df_plot, f'compliance_score{judge_identifier}', y_range=y_range, ax=axes[0,2], title_addon="")
gran_5_range = [3,5]
gran_10_range = [7,10]
row_heat_map(judge_identifier="_mistral_g5", y_range=gran_5_range, row_idx=0)
row_heat_map(judge_identifier="_llama_g5", y_range=gran_5_range, row_idx=1)
row_heat_map(judge_identifier="_mistral_g10", y_range=gran_10_range, row_idx=2)
row_heat_map(judge_identifier="_llama_g10", y_range=gran_10_range, row_idx=3)
None
fig.suptitle("Mistral (first row) and LLaMA (second row) scores")